In [1]:
import vision as vis
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos

In [2]:
get_prox = False
if get_prox == False:
    state = 0

if get_prox == True:
    state = 1

In [4]:
# Main -> state machine

obstacle = False
path_calculated = False

cap = cv.VideoCapture(0)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while(1):
    time.sleep(0.5)
    captured, img = cap.read()
    if captured:
        cropped, cropped_img = vis.crop_map(img)
        # print(cropped_img.shape)        # uncomment to debug
        # cv.imshow("Original", img)        # uncomment to debug
        cv.imshow("Cropped", cropped_img)
        if cropped:
            im3 = np.zeros(cropped_img.shape)
            found, obstacles = vis.extract_obstacles(cropped_img)
            if found:
                ex_obstacles = vis.expand_obstacles(obstacles)
                cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                cv.imshow("expanded", im3)
            found_rob, robot_pos = pos.get_robot_position(cropped_img)
            if found_rob:
                vis.annotate_robot(robot_pos, im3)
            found_goal, goal_pos = pos.get_goal_position(cropped_img)
            if found_goal:
                vis.annotate_goal(goal_pos, im3)
            found_arch, arch_pos = pos.get_arch_positions(cropped_img)
            if found_arch:
                vis.annotate_arch(arch_pos, im3)
            if (found and found_rob and found_goal and found_arch):
                if path_calculated == False:
                    vertices = vis.convert_vertice(ex_obstacles)
                    pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(goal_pos))
                    path, ignore1,ignore2,ignore3 =\
                    short.names_to_subpaths(pathname,vertices,tuple(robot_pos[0]), tuple(goal_pos))
                    print("path", path)
                    vis.annotate_path(path,im3)
                    state = 0
                    cv.imshow("environment", im3)
                    time.sleep(1)
                    break
            cv.imshow("environment", im3)
    else:
        print("There was a problem in the capture")
        break
    if (cv.waitKey(1) & 0xFF) == ord('q'):                       # quit when 'Q' key is pressed
        break
cv.imshow("environment",im3)
cap.release()
# cv.destroyAllWindows()
    

"""
# robot run    
elif short.find_shortest_path() == True:
    state = 1

obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
"""
            
            
        
    

no circle found
None
no circle found
no circle found
no circle found
no circle found
None
no circle found
[[182 168  20]]

coordinates of circle is: [[182 168  20]] 
number of circles detected is: 1
number of circles found 1
[[182 168  20]]

coordinates of circle is: [[182 168  20]] 
number of circles detected is: 1
number of circles found 1
[[182 168  20]]

coordinates of circle is: [[182 168  20]] 
number of circles detected is: 1
number of circles found 1
[[190 214  14]
 [180 166  20]]

coordinates of circle is: [[190 214  14]
 [180 166  20]] 
number of circles detected is: 2
-48 10
Start (180, 166)
Goal (310, 132)
Vertices [[(116, 246), (262, 336), (238, 380), (90, 289), (269, 197), (351, 209), (332, 314), (250, 297)], [(116, 246), (262, 336), (238, 380), (90, 289), (269, 197), (351, 209), (332, 314), (250, 297)]]
list neighbours [[(17, 134.3726162579266)], [(9, 0.0)], [(10, 0.0)], [(11, 0.0)], [(12, 0.0)], [(13, 0.0)], [(14, 0.0)], [(15, 0.0)], [(16, 0.0)], [(1, 0.0)], [(2, 0.0)],

'\n# robot run    \nelif short.find_shortest_path() == True:\n    state = 1\n\nobstacle = check_obstacle(prox_horizontal)\n    \n        # global navigation\n        if  obstacle == False:\n            motion_control\n\n        # local navigation\n        elif obstacle == True:\n'

In [ ]:
OBSTACLE_THRESHOLD 10

def check_obstacle(prox_horizontal):
    prox_values = [prox_horizontal[0], prox_horizontal[4]]
    if prox_values[0] > OBSTACLE_THRESHOLD or prox_values[1] > OBSTACLE_THRESHOLD:
        obstacle = True
        
    else:
        obstacle = False
        
        
    